# Automação de Consulta de Processos Jurídicos

## Contexto

Este notebook apresenta um processo de automação de preenchimento de um formulário de pesquisa de processos jurídicos de um site fictício. Essencialmente, o processo envolve duas etapas: escolha de um Estado e preenchimento do formulário de busca de processos.

## Guia do Processo

Para executar esta automação, vamos seguir este passo-a-passo:

1. Abrir um navegador Google Chrome

2. Abrir a página de busca (index.html)

3. Selecionar um estado

4. Mudar de aba

5. Preencher o formulário de busca

6. Enviar o formulário

7. Confirmar o alerta após envio do formulário

8. Aguardar resultado da busca

9. Tomar uma decisão com base no resultado

## Importações

In [1]:
# Importa a biblioteca time
import time

# Importa a biblioteca os
import os

# Importa o pandas
import pandas as pd

# Importa o webdriver
from selenium import webdriver

# Importa o Service para especificarmos o caminho do webdriver
from selenium.webdriver.chrome.service import Service

# Importa o web driver manager para instalarmos o webdriver compatível com a versão do Chrome
from webdriver_manager.chrome import ChromeDriverManager

# Instala o Chrome Driver e fornece o caminho do executável
servico = Service(ChromeDriverManager().install())

# Importa a classe By para localizar elementos
from selenium.webdriver.common.by import By

# Importa o Action Chains para trabalharmos com drop-downs dinâmicos
from selenium.webdriver import ActionChains

# Importa o Alert para trabalharmos com alertas
from selenium.webdriver.common.alert import Alert

# Importa o expected conditions para esperar elementos aparecerem
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Criação do navegador

In [2]:
# Cria um navegdor Google Chrome
navegador = webdriver.Chrome(service=servico)

## Abrir Página de Busca do Estado

In [3]:
# Caminho deste notebook
caminho_nb = os.getcwd()

# Arquivo que desejamos abrir
pagina_busca_estado = 'index.html'

# Caminho completo da página
caminho_pagina_busca_estado = os.path.join(caminho_nb, pagina_busca_estado)

# Abre a página de busca por estado
navegador.get(caminho_pagina_busca_estado)

## Selecionar um Estado

In [4]:
# Localiza o menu
menu_estados = navegador.find_element(By.CSS_SELECTOR, '.dropdown-menu .btn')

# Estado que desejamos selecionar (Rio de Janeiro)
estado = navegador.find_element(By.CSS_SELECTOR, '.dropdown-menu .dropdown-content ul li a')

# Posiciona o mouse sobre o menu
ActionChains(navegador).move_to_element(menu_estados).perform()

# Aguarda 5s
time.sleep(5)

# Clica no estado desejado
estado.click()

## Mudar de Aba

In [5]:
# Muda para a aba do formulário
nova_aba = navegador.window_handles[1]
navegador.switch_to.window(nova_aba)

## Preencher o Formulário de Busca

In [6]:
# Localiza o campo de nome e preenche ele
navegador.find_element(By.ID, 'nome').send_keys('Diego Moura Torres')

# Localiza o campo de advogado e preenche ele
navegador.find_element(By.ID, 'advogado').send_keys('Maria Elza Liz Porto')

# Localiza o campo número do processo e preenche ele
navegador.find_element(By.ID, 'numero').send_keys('10')

## Enviar o Formulário

In [7]:
# Localiza o botão de enviar formulário e clica nele
navegador.find_element(By.CSS_SELECTOR, '#formulario .btn.registerbtn').click()

## Confirmar o Alerta

In [8]:
# Seleciona o alerta e aceita ele
alerta = Alert(navegador)

# Aceita o alerta
alerta.accept()

## Aguardar Resultado e Reagir

In [9]:
# Espera 15s pelo alerta
try:
    # Tenta localizar o alerta
    alerta = WebDriverWait(navegador, 20).until(EC.alert_is_present())

    # Aguarda 1s
    time.sleep(1)

    # Extrai o texto do alerta
    texto_alerta = alerta.text
    
    if texto_alerta:
        if 'Processo encontrado' in texto_alerta:
            alerta.accept()
            print('Processo encontrado!')
        else:
            print('Nenhum processo encontrado!')
    
except:
    print('Alerta não encontrado')

Processo encontrado!
